# IMPORTS and DOWNLOADS

In [45]:
!pip install pyBigWig

In [46]:
import pandas as pd
import os
import pyBigWig
import numpy as np
from notebooks.consts import NOTEBOOK_PATH
from pyprojroot import here
from src.tauso.off_target.Roni.off_target_pipeline.off_target_functions import parse_gtf
from notebooks.consts import SAVED_FEATURES


# FOLDERS

In [66]:
PROJECT_ROOT = here()
DATA_DIR = os.path.join(PROJECT_ROOT, "src", "tauso", "off_target", "Roni", "data")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "src", "tauso", "off_target", "Roni", "outputs")
gtf_path = os.path.join(DATA_DIR, "gencode.v34.chr_patch_hapl_scaff.annotation.gtf")

# DATA

In [67]:
aso_df = pd.read_csv(NOTEBOOK_PATH / "data/data_asoptimizer_updated.csv")
annotations = parse_gtf(gtf_path, "../outputs/gtf_pickle_output_file_path.pkl")

/tmp/ipykernel_83456/723540510.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  aso_df = pd.read_csv(NOTEBOOK_PATH / "data/data_asoptimizer_updated.csv")


Read GTF file


In [68]:
bw = pyBigWig.open("human_unselected_40S.RiboProElong.bw")

In [50]:
bw.chroms()

{'chr1': 248956422,
 'chr2': 242193529,
 'chr3': 198295559,
 'chr4': 190214555,
 'chr5': 181538259,
 'chr6': 170805979,
 'chr7': 159345973,
 'chr8': 145138636,
 'chr9': 138394717,
 'chr10': 133797422,
 'chr11': 135086622,
 'chr12': 133275309,
 'chr13': 114364328,
 'chr14': 107043718,
 'chr15': 101991189,
 'chr16': 90338345,
 'chr17': 83257441,
 'chr18': 80373285,
 'chr19': 58617616,
 'chr20': 64444167,
 'chr21': 46709983,
 'chr22': 50818468,
 'chrX': 156040895,
 'chrY': 57227415}

In [51]:
chrom = "chr1"
start = 629095
end   = 629099

values = bw.values(chrom, start, end)

In [52]:
len(values)
values

[4.0, 1.0, 1.0, nan]

# HELPER FUNCTIONS

In [69]:
def build_gene_lookup_from_transcripts(annotations):
    """
    Build:
    gene_name -> {chrom, gene_start, gene_end, strand}

    Resolves strand conflicts by choosing the longest transcript.
    """

    gene_to_transcripts = {}

    for tx_id, info in annotations.items():
        gene = info["gene_name"]

        start0 = info["start"] - 1  # 0-based
        end0   = info["end"]
        length = end0 - start0

        record = {
            "chrom": info["chrom"],
            "gene_start": start0,
            "gene_end": end0,
            "strand": info["strand"],
            "length": length
        }

        gene_to_transcripts.setdefault(gene, []).append(record)

    gene_lookup = {}

    for gene, records in gene_to_transcripts.items():
        # choose longest transcript
        longest = max(records, key=lambda r: r["length"])

        gene_lookup[gene] = {
            "chrom": longest["chrom"],
            "gene_start": longest["gene_start"],
            "gene_end": longest["gene_end"],
            "strand": longest["strand"]
        }

    return gene_lookup

In [70]:
def add_genomic_coordinates(aso_df, annotations):
    """
    Adds genomic coordinates to aso_df:
    chrom, gene_start, gene_end, target_start, target_end

    Rows with unmappable genes get NaN values.
    """

    gene_lookup = build_gene_lookup_from_transcripts(annotations)

    out = aso_df.copy()

    chroms = []
    gene_starts = []
    gene_ends = []
    target_starts = []
    target_ends = []

    for _, row in out.iterrows():
        gene = row.get("Canonical Gene Name")

        # Handle controls / missing genes
        if not isinstance(gene, str) or gene not in gene_lookup:
            chroms.append(np.nan)
            gene_starts.append(np.nan)
            gene_ends.append(np.nan)
            target_starts.append(np.nan)
            target_ends.append(np.nan)
            continue

        info = gene_lookup[gene]
        chrom = info["chrom"]
        g_start = info["gene_start"]
        g_end = info["gene_end"]
        strand = info["strand"]

        L = int(row["Location_in_sequence"])   # 1-based
        k = int(row["true_length_of_seq"])
        gene_len = g_end - g_start

        if strand == "+":
            t_start = g_start + (L - 1)
        else:
            t_start = g_start + (gene_len - L - k + 1)

        t_end = t_start + k

        chroms.append(chrom)
        gene_starts.append(g_start)
        gene_ends.append(g_end)
        target_starts.append(t_start)
        target_ends.append(t_end)

    out["chrom"] = chroms
    out["gene_start"] = gene_starts
    out["gene_end"] = gene_ends
    out["target_start"] = target_starts
    out["target_end"] = target_ends

    return out

In [71]:
def reduce_values(values, how):
    if values.size == 0:
        return 0.0

    if how == "sum":
        return values.sum()
    elif how == "mean":
        return values.mean()
    elif how == "max":
        return values.max()
    elif how == "nz_mean":
        nz = values[values > 0]
        return nz.mean() if nz.size > 0 else 0.0

In [73]:
def create_ribo_seq_features(bw, aso_df, flanks=(0,10,20,50,100), how="mean"):

    if how not in {"sum", "mean", "max", "nz_mean"}:
        raise ValueError(how)

    required = ["chrom", "target_start", "target_end", "gene_start", "gene_end", "index"]
    df = aso_df.dropna(subset=required).reset_index(drop=True)

    chroms  = df["chrom"].values
    t_start = df["target_start"].astype(int).values
    t_end   = df["target_end"].astype(int).values
    g_start = df["gene_start"].astype(int).values
    g_end   = df["gene_end"].astype(int).values
    indices = df["index"].values

    features = {}

    for i in range(len(df)):
        chrom = chroms[i]
        chrom_len = bw.chroms()[chrom]

        feat = {}

        # ---------- gene ----------
        gs = max(0, g_start[i])
        ge = min(g_end[i], chrom_len)

        if ge > gs:
            values = np.nan_to_num(
                np.array(bw.values(chrom, gs, ge), dtype=float),
                nan=0.0
            )
            feat[f"ribo_gene_{how}"] = reduce_values(values, how)
        else:
            feat[f"ribo_gene_{how}"] = 0.0

        # ---------- flanks ----------
        for f in flanks:
            s = max(0, t_start[i] - f)
            e = min(t_end[i] + f, chrom_len)

            if e > s:
                values = np.nan_to_num(
                    np.array(bw.values(chrom, s, e), dtype=float),
                    nan=0.0
                )
                val = reduce_values(values, how)
            else:
                val = 0.0

            feat[f"ribo_f{f}_{how}"] = val

        features[indices[i]] = feat

    return features

In [78]:
def save_ribo_seq_features(feature_dict, feature_dir):
    """
    Save ribo-seq features to CSV files.
    One CSV per feature:
        index, feature_value
    """
    os.makedirs(feature_dir, exist_ok=True)

    # Collect all feature names
    feature_names = set()
    for feats in feature_dict.values():
        feature_names.update(feats.keys())

    # Build one dataframe per feature
    for feature in sorted(feature_names):
        rows = []

        for aso_idx, feats in feature_dict.items():
            rows.append({
                "index": aso_idx,
                feature: feats.get(feature, np.nan)  # <-- FIX
            })

        df = pd.DataFrame(rows)
        df.sort_values("index", inplace=True)

        out_path = os.path.join(feature_dir, f"{feature}.csv")
        df.to_csv(out_path, index=False)

# EXECUTION

In [74]:
new = add_genomic_coordinates(aso_df, annotations)

In [75]:
features = create_ribo_seq_features(bw, new)

In [76]:
features

{0: {'ribo_gene_mean': 0.0036774363015497765,
  'ribo_f0_mean': 0.1875,
  'ribo_f10_mean': 0.08333333333333333,
  'ribo_f20_mean': 0.05357142857142857,
  'ribo_f50_mean': 0.0603448275862069,
  'ribo_f100_mean': 0.06481481481481481},
 1: {'ribo_gene_mean': 0.0036774363015497765,
  'ribo_f0_mean': 0.0,
  'ribo_f10_mean': 0.027777777777777776,
  'ribo_f20_mean': 0.017857142857142856,
  'ribo_f50_mean': 0.008620689655172414,
  'ribo_f100_mean': 0.013888888888888888},
 2: {'ribo_gene_mean': 0.0036774363015497765,
  'ribo_f0_mean': 0.0625,
  'ribo_f10_mean': 0.08333333333333333,
  'ribo_f20_mean': 0.05357142857142857,
  'ribo_f50_mean': 0.06896551724137931,
  'ribo_f100_mean': 0.037037037037037035},
 3: {'ribo_gene_mean': 0.0036774363015497765,
  'ribo_f0_mean': 0.0625,
  'ribo_f10_mean': 0.027777777777777776,
  'ribo_f20_mean': 0.017857142857142856,
  'ribo_f50_mean': 0.008620689655172414,
  'ribo_f100_mean': 0.013888888888888888},
 4: {'ribo_gene_mean': 0.0036774363015497765,
  'ribo_f0_me

In [79]:
save_ribo_seq_features(features, "temp_feat")